## Imports

In [1]:
# set the matplotlib backend so figures can be saved in the background
import matplotlib
matplotlib.use("Agg")

# import the necessary packages
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.pooling import AveragePooling2D
from keras.applications import ResNet50
from keras.layers.core import Dropout
from keras.layers.core import Flatten
from keras.layers.core import Dense
from keras.layers import BatchNormalization, Activation
from keras.layers import Input
from keras.models import Model
from keras.initializers import glorot_uniform
from keras.optimizers import SGD, Adam
from keras import regularizers
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imutils import paths # Se usa para levantar a una lista todas la imagenes de un path
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from keras.models import load_model
from collections import deque
import argparse
import pickle
import argparse
import pickle
import cv2
import os
from datetime import datetime

Using TensorFlow backend.


In [2]:
def make_dir(directory):
        if os.path.exists(directory):
            shutil.rmtree(directory)
        os.makedirs(directory)

## Arguments

In [3]:
path_dataset = '/media/sf_Google_Drive/LudisAI/01 - Sport Classifier/Datasets/'
path_labels = '/media/sf_Google_Drive/LudisAI/01 - Sport Classifier/'
path_plots = '/media/sf_Google_Drive/LudisAI/01 - Sport Classifier/Metricas/'
path_model = '/media/sf_Google_Drive/LudisAI/01 - Sport Classifier/'

seed = 1984

## Create Dataset

In [ ]:
# initialize the set of labels from the sports activity dataset we are
# going to train our network on
LABELS = set(["badminton", "tennis", "squash", "table_tennis", "padel", "otros"])

# grab the list of images in our dataset directory, then initialize
# the list of data (i.e., images) and class images
print("[INFO] loading images...")
data = []
labels = []

# loop over the image paths for each label
for sport_label in LABELS:
    imagePaths = list(paths.list_images(path_dataset + sport_label))
    count = 0
    tot_count = len(imagePaths)
    print('Loading images for ', sport_label)
    for imagePath in imagePaths:
        # load the image, convert it to RGB channel ordering, and resize
        # it to be a fixed 224x224 pixels, ignoring aspect ratio
        image = cv2.imread(imagePath)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = cv2.resize(image, (224, 224))

        # update the data and labels lists, respectively
        data.append(image)
        if sport_label == 'squash':
            labels.append(sport_label)
        else:
            labels.append('no_squash')
        
        deciles = [int(x) for x in np.linspace(0,1,11)*tot_count]
        if count in deciles:
            print('Imagenes procesadas: {} de {} ({}%)'.format(count, tot_count, np.round(count*100/tot_count, 0)))
        count += 1

In [ ]:
## Export Dataset
np.savez(path_dataset + 'sports_data_binario.npz', np.array(data))
np.savez(path_dataset + 'sports_data_labels_binario.npz', np.array(labels))

### Load Dataset and Train/Test Split

In [ ]:
data = np.load(path_dataset + 'sports_data_binario.npz')['arr_0']
labels = np.load(path_dataset + 'sports_data_labels_binario.npz')['arr_0']

In [ ]:
labels.shape

In [ ]:
# perform one-hot encoding on the labels
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
# partition the data into training and testing splits using 75% of
# the data for training and the remaining 25% for testing
X_train, X_test, Y_train, Y_test = train_test_split(data, labels, test_size=0.25, 
                                                    stratify=labels, random_state=seed)

In [ ]:
print(X_train.shape)
print(Y_train.shape)

In [ ]:
print('Registros Train: ', len(X_train))
print('Registros Test: ', len(X_test))

In [ ]:
## Agregado Imagenes test (sacadas con baja calidad)
imagePaths = list(paths.list_images(path_dataset + 'fotos_test'))

test_data = []
test_labels = []

for imagePath in imagePaths:
    # load the image, convert it to RGB channel ordering, and resize
    # it to be a fixed 224x224 pixels, ignoring aspect ratio
    image = cv2.imread(imagePath)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (224, 224))

    # update the data and labels lists, respectively
    test_data.append(image)
    test_labels.append('squash')

In [ ]:
X_test = np.append(X_test, test_data, axis=0)
print(X_test.shape)

In [ ]:
test_labels = lb.fit_transform(test_labels)
Y_test = np.append(Y_test, test_labels, axis=0)
print(Y_test.shape)

In [ ]:
print('Registros Train: ', len(X_train))
print('Registros Test: ', len(X_test))

In [ ]:
## Export Train/Test Sets
np.savez(path_dataset + 'X_train_squash.npz', X_train)
np.savez(path_dataset + 'X_test_squash.npz', X_test)
np.savez(path_dataset + 'Y_train_squash.npz', Y_train)
np.savez(path_dataset + 'Y_test_squash.npz', Y_test)

In [ ]:
X_train = np.load(path_dataset + 'X_train_squash.npz')
Y_train = np.load(path_dataset + 'Y_train_squash.npz')
X_test = np.load(path_dataset + 'X_test_squash.npz')
Y_test = np.load(path_dataset + 'Y_test_squash.npz')

In [ ]:
print(X_test.shape)
print(Y_test.shape)

## Data Augmentation

In [ ]:
# initialize the training data augmentation object
trainAug = ImageDataGenerator(rotation_range=30,
                              zoom_range=0.15,
                              width_shift_range=0.2,
                              height_shift_range=0.2,
                              shear_range=0.15,
                              horizontal_flip=True,
                              fill_mode="nearest")

# initialize the validation/testing data augmentation object (which
# we'll be adding mean subtraction to)
valAug = ImageDataGenerator()

# define the ImageNet mean subtraction (in RGB order) and set the
# the mean subtraction value for each of the data augmentation
# objects
mean = np.array([123.68, 116.779, 103.939], dtype="float32")
trainAug.mean = mean
valAug.mean = mean

## Model

In [ ]:
# load the ResNet-50 network, ensuring the head FC layer sets are left
# off
baseModel = ResNet50(weights="imagenet", include_top=False, input_tensor=Input(shape=(224, 224, 3)))

# construct the head of the model that will be placed on top of the
# the base model
headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(7, 7))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(512, use_bias=False)(headModel)#, kernel_initializer=glorot_uniform(seed=seed)
headModel = BatchNormalization()(headModel)
headModel = Activation("relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(1, activation="sigmoid")(headModel) #, kernel_initializer=glorot_uniform(seed=seed)

# place the head FC model on top of the base model (this will become
# the actual model we will train)
model = Model(inputs=baseModel.input, outputs=headModel)

# loop over all layers in the base model and freeze them so they will
# *not* be updated during the training process
for layer in baseModel.layers:
    layer.trainable = False

In [ ]:
with open(path_model + 'model_summary.txt','w') as fh:
    # Pass the file handle in as a lambda function to make it callable
    model.summary(print_fn=lambda x: fh.write(x + '\n'))

### Training

In [ ]:
epochs = 20
batch_size = 64

In [ ]:
start_time = datetime.now()
# compile our model (this needs to be done after our setting our
# layers to being non-trainable)
print("[INFO] compiling model...")
#opt = Adam(learning_rate=0.0001, beta_1=0.9, beta_2=0.999, amsgrad=False)
model.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])

# train the head of the network for a few epochs (all other layers
# are frozen) -- this will allow the new FC layers to start to become
# initialized with actual "learned" values versus pure random
print("[INFO] training head...")
H = model.fit_generator(trainAug.flow(X_train, Y_train, batch_size=batch_size),
                        steps_per_epoch=len(X_train) // batch_size,
                        validation_data=valAug.flow(X_test, Y_test),
                        validation_steps=len(X_test) // batch_size,
                        epochs=epochs)

duration = datetime.now()-start_time
print('Duracion: ', duration)

### Evaluation

In [ ]:
# evaluate the network
print("[INFO] evaluating network...")
#predictions = model.predict(X_test, batch_size=batch_size)
predictions_class = (predictions>0.5)
print(classification_report(Y_test, predictions_class))

# plot the training loss and accuracy
N = epochs #args["epochs"]
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), H.history["acc"], label="train_acc")
plt.plot(np.arange(0, N), H.history["val_acc"], label="val_acc")
plt.title("Training Loss and Accuracy on Dataset")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.savefig(path_plots + 'loss_accuracy_squash_classifier_plot.png')

In [ ]:
import pandas as pd
metricas = pd.DataFrame(classification_report(Y_test, predictions_class, output_dict=True)).transpose()
metricas.to_csv(path_plots + 'metrics_squash_binario.csv', sep=';', decimal='.', encoding='utf-8')

### Model Export

In [ ]:
# serialize the model to disk
print("[INFO] serializing network...")
model.save(path_model + 'squash_classifier.h5')

# serialize the label binarizer to disk
f = open(path_model + 'label_binarizer', "wb")
f.write(pickle.dumps(lb))
f.close()

### Video Classification

In [ ]:
input_video = path_dataset + 'video_test/Ludis - Deportes Raqueta.mp4'
output_video = path_dataset + 'video_test/Ludis - Deportes Raqueta_model_binario.mp4'

In [ ]:
start_time = datetime.now()

# load the trained model and label binarizer from disk
print("[INFO] loading model and label binarizer...")
model = load_model(path_model + 'squash_classifier')
lb = pickle.loads(open(path_model + 'label_binarizer', "rb").read())

# initialize the image mean for mean subtraction along with the
# predictions queue
mean = np.array([123.68, 116.779, 103.939][::1], dtype="float32")
Q = deque(maxlen=128)

# initialize the video stream, pointer to output video file, and
# frame dimensions
vs = cv2.VideoCapture(input_video)
writer = None
(W, H) = (None, None)

# loop over frames from the video file stream
while True:
    # read the next frame from the file
    (grabbed, frame) = vs.read()

    # if the frame was not grabbed, then we have reached the end
    # of the stream
    if not grabbed:
        break

    # if the frame dimensions are empty, grab them
    if W is None or H is None:
        (H, W) = frame.shape[:2]

    # clone the output frame, then convert it from BGR to RGB
    # ordering, resize the frame to a fixed 224x224, and then
    # perform mean subtraction
    output = frame.copy()
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame = cv2.resize(frame, (224, 224)).astype("float32")
    frame -= mean

    # make predictions on the frame and then update the predictions
    # queue
    preds = model.predict(np.expand_dims(frame, axis=0))[0]
    Q.append(preds)

    # perform prediction averaging over the current history of
    # previous predictions
    results = np.array(Q).mean(axis=0)
    i = np.argmax(results)
    label = lb.classes_[i]

    # draw the activity on the output frame
    text = "Deporte: {}".format(label)
    cv2.putText(output, text, (35, 50), cv2.FONT_HERSHEY_SIMPLEX,
        1.25, (0, 255, 0), 2)

    # check if the video writer is None
    if writer is None:
        # initialize our video writer
        fourcc = cv2.VideoWriter_fourcc(*"MJPG")
        writer = cv2.VideoWriter(output_video, fourcc, 30,
            (W, H), True)

    # write the output frame to disk
    writer.write(output)

    # show the output image
    #cv2.imshow("Output", output)
    key = cv2.waitKey(1) & 0xFF

    # if the `q` key was pressed, break from the loop
    if key == ord("q"):
        break

# release the file pointers
print("[INFO] cleaning up...")
writer.release()
vs.release()

duration = datetime.now()-start_time
print('Duracion: ', duration)

### Video Classification - Sampling Frames

In [ ]:
input_video = path_dataset + 'video_test/squash_beginner.mp4'

In [ ]:
start_time = datetime.now()

# load the trained model and label binarizer from disk
print("[INFO] loading model and label binarizer...")
#model = load_model(path_model + 'sports_classifier')
#lb = pickle.loads(open(path_model + 'label_binarizer', "rb").read())

# initialize the image mean for mean subtraction along with the
# predictions queue
mean = np.array([123.68, 116.779, 103.939][::1], dtype="float32")

# initialize the video stream, pointer to output video file, and
# frame dimensions
vs = cv2.VideoCapture(input_video)

q_frames = int(vs.get(cv2.CAP_PROP_FRAME_COUNT))
frames_predict = np.random.choice(range(1, q_frames), size=int(q_frames*0.25), replace=False)
q_frames_predict = len(frames_predict)

pred_list = []
count = 0
# loop over frames from the video file stream
print("[INFO] classifying video...")
for frame2read in frames_predict:
    # Set video frame to read
    vs.set(1, frame2read)
    
    # Read the next frame from the file
    (grabbed, frame) = vs.read()

    # Convert frame from BGR to RGB, resize the frame to a fixed 224x224, and then
    # perform mean subtraction
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame = cv2.resize(frame, (224, 224)).astype("float32")
    frame -= mean

    # Make predictions on the frame and then update the predictions list
    # expand_dims con axis=0 agrega la dimension channels_first
    preds = model.predict(np.expand_dims(frame, axis=0))[0]
    pred_list.append(preds)
    
    deciles = [int(x) for x in np.linspace(0,1,11)*q_frames_predict]
    if count in deciles:
        print('Processed frames: {} de {} ({}%)'.format(count, q_frames_predict, np.round(count*100/q_frames_predict, 0)))
    count += 1

# release the file pointers
print("[INFO] cleaning up...")
vs.release()

print("[INFO] averaging predictions...")
results = np.array(pred_list).mean(axis=0)
i = np.argmax(results)
label = lb.classes_[i]

duration = datetime.now()-start_time
print('Duracion: ', duration)
print('El video es de ', label)

In [ ]:
(np.array(pred_list)>0.5).sum() / len(pred_list)

## Test

In [4]:
model = load_model(path_model + 'squash_classifier.h5')
lb = pickle.loads(open(path_model + 'label_binarizer', "rb").read())

In [6]:
imagePaths = list(paths.list_images(path_dataset + 'fotos_test'))
image_count = len(imagePaths)

mean = np.array([123.68, 116.779, 103.939][::1], dtype="float32")
count = 0

images = []
pred_list = []

print("[INFO] classifying images...")
for imagePath in imagePaths:

    # Convert frame from BGR to RGB, resize the frame to a fixed 224x224, and then
    # perform mean subtraction
    image = cv2.imread(imagePath)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (224, 224)).astype("float32")
    image -= mean
    images.append(image)

    # Make predictions on the frame and then update the predictions list
    # expand_dims con axis=0 agrega la dimension channels_first
    preds = model.predict(np.expand_dims(image, axis=0))[0]
    i = int(np.squeeze(preds)>0.5)
    label = lb.classes_[i]
    pred_list.append(label)
    
    deciles = [int(x) for x in np.linspace(0,1,11)*image_count]
    if count in deciles:
        print('Processed images: {} de {} ({}%)'.format(count, image_count, np.round(count*100/image_count, 0)))
    count += 1

[INFO] classifying images...
Processed images: 0 de 377 (0.0%)
Processed images: 37 de 377 (10.0%)
Processed images: 75 de 377 (20.0%)
Processed images: 113 de 377 (30.0%)
Processed images: 150 de 377 (40.0%)
Processed images: 188 de 377 (50.0%)
Processed images: 226 de 377 (60.0%)
Processed images: 263 de 377 (70.0%)
Processed images: 301 de 377 (80.0%)
Processed images: 339 de 377 (90.0%)


In [14]:
print('Test Accuracy: ', np.round((pd.Series(pred_list) == 'squash').sum()/len(pred_list)*100,2))

Test Accuracy:  87.0


In [ ]:
## Check misclassified images
for i in range(len(images)):
    img = images[i]
    label = pred_list[i]
    if label == 'no_squash':
        cv2.imshow('Mal clasificada', img)

        k = cv2.waitKey(1) # Parametriza la cantidad de milisegundos que muestra la foto
        if k == 27: # si se apreta la tecla ESC
            break

cv2.destroyAllWindows()